In [1]:
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from public_data.models import Ocsge, Departement, CouvertureSol, UsageSol, OcsgeDiff, CouvertureUsageMatrix
from django.db import connection
from django.db.models import Subquery, OuterRef, DecimalField, CharField, F, Value
from django.db.models.functions import Cast
from django.contrib.gis.db.models.functions import Area, Transform


params = [0.3658103942871094,43.646877927093804,0.6255340576171876,43.72917630660924, 2016]

In [2]:
query = (
    "SELECT "
    "o.id, o.couverture_label, o.usage_label, o.millesime, t.map_color, "
    "o.year, st_AsGeoJSON(o.mpoly, 8) AS geojson "
    "FROM public_data_ocsge o "
    "INNER JOIN public_data_couverturesol t ON t.code_prefix = o.couverture "
    "WHERE o. mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) "
    "AND o.year = %s"
)
with connection.cursor() as cursor:
    cursor.execute(query, params)
    rows = cursor.fetchall()
len(rows)

11061

In [3]:
qs = Ocsge.objects.all()
qs = qs.annotate(area=Cast(Area(Transform("mpoly", 2154)), DecimalField(max_digits=15, decimal_places=4)))

qs.get(id=2500).area

Decimal('95.1684')

In [4]:
OcsgeDiff.objects.all().count()

17239

In [5]:
ocsge_diff = OcsgeDiff.objects.get(pk=1000)
ocsge_diff

<OcsgeDiff: OcsgeDiff object (1000)>

In [6]:
CouvertureUsageMatrix.objects.filter(
    couverture__code_prefix=ocsge_diff.cs_old,
    usage__code_prefix=ocsge_diff.us_old
).count()

1

In [7]:
CouvertureUsageMatrix.objects.filter(
    couverture__code_prefix=ocsge_diff.cs_old,
    usage__code_prefix=ocsge_diff.us_old
).get().is_artificial

False

In [8]:
CouvertureUsageMatrix.objects.filter(
    couverture__code_prefix=ocsge_diff.cs_new,
    usage__code_prefix=ocsge_diff.us_new
).get().is_artificial

True

In [9]:
qs = OcsgeDiff.objects.annotate(
    old_is_art=Subquery(
        CouvertureUsageMatrix.objects.filter(
            couverture__code_prefix=OuterRef("cs_old"),
            usage__code_prefix=OuterRef("us_old")
        ).values("is_artificial")[:1]
    )
).annotate(
    new_is_art=Subquery(
        CouvertureUsageMatrix.objects.filter(
            couverture__code_prefix=OuterRef("cs_new"),
            usage__code_prefix=OuterRef("us_new")
        ).values("is_artificial")[:1]
    )
).annotate(
    is_new_artif_2=Value(F("new_is_art") and not F("old_is_art"))
)

print(qs.last().old_is_art)
print(qs.last().new_is_art)
print(qs.last().is_new_artif_2)

print(qs.first().old_is_art)
print(qs.first().new_is_art)
print(qs.first().is_new_artif_2)

False
True
False
True
True
False


In [10]:
print(qs.first().cs_new)
print(qs.first().us_new)
print(qs.first().new_is_art)

CS1.1.2.1
US4.1.2
True


In [11]:
CouvertureUsageMatrix.objects.filter(
    couverture__code_prefix="CS2.2.1",
    usage__code_prefix="US1.1"
).values("is_artificial")[:1]


<QuerySet [{'is_artificial': False}]>

In [12]:
o = qs.filter(id=2500).first()
print("OLD :", o.us_old, o.cs_old, o.old_is_art)
print("NEW :", o.us_new, o.cs_new, o.new_is_art)
print("Test :", o.new_is_art and not o.old_is_art)
print(f"is_new_artif={o.is_new_artif_2}")

OLD : US235 CS2.2.1 True
NEW : US235 CS1.1.1.2 True
Test : False
is_new_artif=False


In [13]:
# Lister les diff avec CS et US null
OcsgeDiff.objects.filter(cs_new__isnull=True).count()
OcsgeDiff.objects.filter(us_new__isnull=True).count()
OcsgeDiff.objects.filter(cs_old__isnull=True).count()
OcsgeDiff.objects.filter(us_old__isnull=True).count()
OcsgeDiff.objects.filter(cs_old__isnull=True, us_old__isnull=True).count()
OcsgeDiff.objects.filter(cs_new__isnull=True, us_new__isnull=True).count()

7

In [14]:
OcsgeDiff.objects.update(
    is_new_artif=Value(
        Subquery(
            CouvertureUsageMatrix.objects.filter(
                couverture__code_prefix=OuterRef("cs_new"),
                usage__code_prefix=OuterRef("us_new")
            ).values("is_artificial")[:1]
        ) and not Subquery(
            CouvertureUsageMatrix.objects.filter(
                couverture__code_prefix=OuterRef("cs_old"),
                usage__code_prefix=OuterRef("us_old")
            ).values("is_artificial")[:1]
        )
    )
)

print(qs.last().is_new_artif_2)


False


In [15]:
query = f"""SELECT m_old.is_artificial
FROM
    public_data_ocsgediff o
    INNER JOIN public_data_couverturesol cs_old ON o.cs_old = cs_old.code_prefix
    INNER JOIN public_data_usagesol us_old ON o.us_old = us_old.code_prefix
    INNER JOIN public_data_couvertureusagematrix m_old 
        ON m_old.couverture_id = cs_old.id 
        AND m_old.usage_id = us_old.id
LIMIT 10
"""

query

'SELECT m_old.is_artificial\nFROM\n    public_data_ocsgediff o\n    INNER JOIN public_data_couverturesol cs_old ON o.cs_old = cs_old.code_prefix\n    INNER JOIN public_data_usagesol us_old ON o.us_old = us_old.code_prefix\n    INNER JOIN public_data_couvertureusagematrix m_old \n        ON m_old.couverture_id = cs_old.id \n        AND m_old.usage_id = us_old.id\nLIMIT 10\n'

In [16]:
UsageSol._meta.db_table

'public_data_usagesol'